In [1]:
from hashlib import sha256
import openpyxl
import time

In [2]:
# Menambahkan transaksi mining dari COINBASE ke miner pada blok yang berusaha ditambahkan
wb = openpyxl.load_workbook('newblock.xlsx')
ws = wb.worksheets[0]
inp = "COINBASE"
outp = [[1050044712949807662371953614880006555363, 25]]
rows = [["COINBASE", str(outp), sha256((str(inp)+str(outp)).encode('utf8')).hexdigest()]]
for row in rows:
    ws.append(row)
wb.save('newblock.xlsx')

In [3]:
# Threshold integer dari hash
threshold = 2**250

In [4]:
# Mencari nonce
nonce = 0
isTrue = False
start_time = time.time()
while isTrue == False:
    wb = openpyxl.load_workbook('newblock.xlsx')
    ws = wb.worksheets[0]
    ws['B1'] = nonce
    wb.save('newblock.xlsx')
    with open("newblock.xlsx", "rb") as f:
        bytes = f.read() # read entire file as bytes
        block_hash = sha256(bytes).hexdigest()
    isTrue = int(block_hash, 16) < threshold
    nonce += 1
end_time = time.time()
nonce = nonce - 1
print("Your block wins with nonce = " + str(nonce) + " in " + str((end_time - start_time)/60) + " minutes ")

Your block wins with nonce = 95 in 0.04069372415542603 minutes 


In [5]:
from os import rename
from os import path
from os import getcwd

In [6]:
# Mengubah nama file newblock.xlsx menjadi hash block tersebut
oldfile = path.join(getcwd(), "newblock.xlsx")
newfile = path.join(getcwd(), block_hash+".xlsx")
rename(oldfile, newfile)

In [7]:
import xlrd

wb = xlrd.open_workbook(block_hash + ".xlsx")
sheet = wb.sheet_by_index(0)
printed_block = "\n" + "Block of hash " + block_hash + "\n" + "\n"
printed_block += "Hash of previous block: " + sheet.row_values(0)[0] + "\n"
printed_block += "Nonce: " + str(sheet.row_values(0)[1]) + "\n" + "\n"
for row in range(1, sheet.nrows):
    printed_block += "Transaction " + str(row) + "\n"
    printed_block += "From: " + str(sheet.row_values(row)[0]) + "\n"
    printed_block += "To & Amount: " + str(sheet.row_values(row)[1]) + "\n"
    printed_block += "Hash: " + sheet.row_values(row)[2] + "\n" + "\n"
print(printed_block)


Block of hash 01d1935e490945ce774c048ff38bd3a56a3fcb57f9045a23340cfb6d4ce52355

Hash of previous block: 131dbbce60547c93b8fdb80b973c62f0c471619a44ed46ed711f1470f7c6d0cc
Nonce: 95.0

Transaction 1
From: 793833148500065718802366039211618163182
To & Amount: [[498063637122669310186914979135247239947, 35], [793833148500065718802366039211618163182, 15]]
Hash: 3114d9838781b5d2ef9b9abaee6bc7e1531345098fcf109d7968e8710392c377

Transaction 2
From: COINBASE
To & Amount: [[1050044712949807662371953614880006555363, 25]]
Hash: 1bbdfaaa54418b4a4f68e3287353d9ffee46d12f1af48e019769c1ec440cb156




In [8]:
# Menambahkan transaksi mining di mana miner menerima 25 koin ke unspent transactions
wb = openpyxl.load_workbook('unspent.xlsx')
ws = wb.worksheets[0]
rows = [[sha256((str(inp)+str(outp)).encode('utf8')).hexdigest(), str(outp)]]
for row in rows:
    ws.append(row)
wb.save('unspent.xlsx')

In [9]:
# Membuat blok kosong selanjutnya yang berisi hash blok yang berhasil ditambahkan di atas
book = openpyxl.Workbook()
sheet = book.active

sheet['A1'] = block_hash

book.save("newblock.xlsx")